##### Dataset properties:
- Training set: 10683 records
- Testing set: 2671 records

##### Features:
- Airline: The name of the airline.

- Date_of_Journey: The date of the journey

- Source: The source from which the service begins.

- Destination: The destination where the service ends.

- Route: The route taken by the flight to reach the destination.

- Dep_Time: The time when the journey starts from the source.

- Arrival_Time: Time of arrival at the destination.

- Duration: Total duration of the flight.

- Total_Stops: Total stops between the source and destination.

- Additional_Info: Additional information about the flight

- Price: The price of the ticket

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/flight-fare-prediction-mh'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Exploring dataset

In [ ]:
!pip install openpyxl

In [ ]:
train = pd.read_excel('../input/flight-fare-prediction-mh/Data_Train.xlsx')
test = pd.read_excel('../input/flight-fare-prediction-mh/Test_set.xlsx')

In [ ]:
train.head()

- we can see columns with details about a particular flight which include date of journey, source, destination, route, departure time, arrival time, duration, total stops(if connected), additional info(regarding baggage, food, etc) and price.
- here we are predicting the price of the ticket using the remaining attributes.

In [ ]:
train.info()

In [ ]:
test.info()

- training data has some missing values in it. So, lets check them.

In [ ]:
is_NaN = train.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = train[row_has_NaN]

print(rows_with_NaN)

From the above cell, we can say that the `null` values are appeared in the same row. So, we can delete that row.

In [ ]:
train.dropna(inplace=True)

let's cross check again

In [ ]:
train.info()

we don't have any missing values now, therefore we can continue further.

- now lets check if we have any duplicate value, so that we can delete the repeated row and then keep the rows that appeared first in the dataset.

In [ ]:
train[train.duplicated()]

In [ ]:
# dropping duplicates value
train.drop_duplicates(keep='first',inplace=True)

Now let's find what are the different values we have in column `Additional_Info`.

In [ ]:
train['Additional_Info'].value_counts()

We can see that few of the values are just differently capitalised. Let's replace one value with other.
- here `No info` and `No Info` are same.

In [ ]:
train['Additional_Info'] = train['Additional_Info'].replace({'No Info':'No info'})

Let's start feature engineering

- first converting duration into minutes
- where ever there is `h` in duration, we will replace it with `*60` and `m` with `*1` and the space between them into `+` to apply `eval` function for finding total minutes.

In [ ]:
# convert duration into minutes format from hours
train['Duration'] = train['Duration'].str.replace('h','*60').str.replace('m','*1').str.replace(' ','+').apply(eval)
train['Duration']

In [ ]:
# do the same for test dataframe
test['Duration'] = test['Duration'].str.replace('h','*60').str.replace('m','*1').str.replace(' ','+').apply(eval)
test['Duration']

Lets break date of journey to parts `month` and `day`.
Dropping the `data of journey` column.

In [ ]:
# Date_of_Journey
train["Journey_day"] = train['Date_of_Journey'].str.split('/').str[0].astype(int)
train["Journey_month"] = train['Date_of_Journey'].str.split('/').str[1].astype(int)
train.drop(["Date_of_Journey"], axis = 1, inplace = True)

test["Journey_day"] = test['Date_of_Journey'].str.split('/').str[0].astype(int)
test["Journey_month"] = test['Date_of_Journey'].str.split('/').str[1].astype(int)
test.drop(["Date_of_Journey"], axis = 1, inplace = True)

Lets get departure hour and departure minute from the departure time column

In [ ]:
# Dep_Time
train["Dep_hour"] = pd.to_datetime(train["Dep_Time"]).dt.hour
train["Dep_min"] = pd.to_datetime(train["Dep_Time"]).dt.minute
train.drop(["Dep_Time"], axis = 1, inplace = True)

# Dep_Time
test["Dep_hour"] = pd.to_datetime(test["Dep_Time"]).dt.hour
test["Dep_min"] = pd.to_datetime(test["Dep_Time"]).dt.minute
test.drop(["Dep_Time"], axis = 1, inplace = True)

lets do the same for arrival time

In [ ]:
train["Arrival_hour"] = pd.to_datetime(train.Arrival_Time).dt.hour
train["Arrival_min"] = pd.to_datetime(train.Arrival_Time).dt.minute
train.drop(["Arrival_Time"], axis = 1, inplace = True)

test["Arrival_hour"] = pd.to_datetime(test.Arrival_Time).dt.hour
test["Arrival_min"] = pd.to_datetime(test.Arrival_Time).dt.minute
test.drop(["Arrival_Time"], axis = 1, inplace = True)

In [ ]:
train.head()

lets do some visualisation of data we have

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize = (15, 10))
plt.title('Count of flights month wise')
ax=sns.countplot(x = 'Journey_month', data = train)
plt.xlabel('Month')
plt.ylabel('Count of flights')
for p in ax.patches:
    ax.annotate(int(p.get_height()), (p.get_x()+0.25, p.get_height()+1), va='bottom',
                    color= 'black')

lets convert `total stops` into numerical form

In [ ]:
train['Total_Stops'].replace(['1 stop', 'non-stop', '2 stops', '3 stops', '4 stops'], [1, 0, 2, 3, 4], inplace=True)
test['Total_Stops'].replace(['1 stop', 'non-stop', '2 stops', '3 stops', '4 stops'], [1, 0, 2, 3, 4], inplace=True)

lets see about `airline` column now

In [ ]:
train['Airline'].value_counts()

Lets visualise it, and find which are effective for the model and which are not

In [ ]:
plt.figure(figsize = (30, 10))
plt.title('Count of flights Airline wise')
ax=sns.countplot(x = 'Airline', data = train)
plt.xlabel('Airline')
plt.ylabel('Count of flights')
for p in ax.patches:
    ax.annotate(int(p.get_height()), (p.get_x()+0.25, p.get_height()+1), va='bottom',
                    color= 'black')

We can see airline services like **Vistara Premium Economy**, **Jet Airways Busniness**, **Multiple Carriers Premium Economy**, **Trujet** as very less flights compared to the other airline services.
Lets label as `other` in the dataframe.
Also, lets check if the price of the these airlines effect the model significantly.

In [ ]:
plt.figure(figsize = (15, 10))
plt.title('Price VS Airlines')
plt.scatter(train['Airline'], train['Price'])
plt.xticks(rotation = 90)
plt.xlabel('Airline')
plt.ylabel('Price of ticket')
plt.xticks(rotation = 90)

The ticket rate for Jet Airways Business airline is high.

In [ ]:
train["Airline"].replace({'Multiple carriers Premium economy':'Other', 
                                                        'Jet Airways Business':'Other',
                                                        'Vistara Premium economy':'Other',
                                                        'Trujet':'Other'
                                                   },    
                                        inplace=True)

test["Airline"].replace({'Multiple carriers Premium economy':'Other', 
                                                        'Jet Airways Business':'Other',
                                                        'Vistara Premium economy':'Other',
                                                        'Trujet':'Other'
                                                   },    
                                        inplace=True)

Lets check how columns, price and additional info are connected

In [ ]:
plt.figure(figsize = (15, 10))
plt.title('Price VS Additional Information')
sns.scatterplot(train['Additional_Info'], train['Price'],data=train)
plt.xticks(rotation = 90)
plt.xlabel('Information')
plt.ylabel('Price of ticket')

We can see columns, Change airports, Business class, 2 Long layover, Red-eye flight and 1 Short layover are less in count. So, convert all of them into others tag.

In [ ]:
train["Additional_Info"].replace({'Change airports':'Others', 'Business class':'Others', '2 Long layover':'Others',
                                  'Red-eye flight':'Others', '1 Short layover':'Others'}, inplace=True)

test["Additional_Info"].replace({'Change airports':'Others', 'Business class':'Others', '2 Long layover':'Others',
                                  'Red-eye flight':'Others', '1 Short layover':'Others'}, inplace=True)

Lets encode categorical data into numerical

In [ ]:
target = train['Price']
train_data = train.drop(["Price"], axis=1)

In [ ]:
numeric_cols = train_data.select_dtypes(include=['int64', 'float64','int32']).columns.tolist()
cat_cols = train_data.select_dtypes(exclude=['int64','float64','int32']).columns.tolist()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()

In [ ]:
train_data[cat_cols] = train_data[cat_cols].apply(encoder.fit_transform)
test[cat_cols] = test[cat_cols].apply(encoder.fit_transform)

In [ ]:
train_data

In [ ]:
X=train_data
y=target

### Building Machine Learning Models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(13 *'*')
print(X_test.shape)
print(y_test.shape)

Let train the models now

#### KNN Model

In [ ]:
# KNN
k_n = list(range(1, 30))
params = dict(n_neighbors = k_n)
knn_regressor = GridSearchCV(KNeighborsRegressor(), params, cv =10, scoring = 'neg_mean_squared_error')
knn_regressor.fit(X_train, y_train)

In [ ]:
y_train_pred =knn_regressor.predict(X_train) ##Predict train result
y_test_pred =knn_regressor.predict(X_test) ##Predict test result

In [ ]:
print("Training Results for KNN Regressor Model:")
print("Root mean squared error: ", (mse(y_train.values, y_train_pred))**(1/2))
print("Mean absolute % error: ", round(mean_absolute_percentage_error(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

#### Decision Tree Model

In [ ]:
depth  =list(range(3,30))
param_grid =dict(max_depth =depth)
tree =GridSearchCV(DecisionTreeRegressor(),param_grid,cv =10)
tree.fit(X_train,y_train)

In [ ]:
dt_train_pred =tree.predict(X_train) ##Predict train result
dt_test_pred =tree.predict(X_test) ##Predict test result

In [ ]:
print("Training Results for Decision Tree Regressor Model:")
print("Root mean squared error: ", (mse(y_train.values, dt_train_pred))**(1/2))
print("Mean absolute % error: ", round(mean_absolute_percentage_error(y_train.values, dt_train_pred)))
print("R-squared: ", r2_score(y_train.values, dt_train_pred))

#### We can there is very less error in Decision Tree Model. So, we will be using Decision Tree Regressor model for the final output.